<font color='IndianRed'>
<font size=10> 

**Identificación de Exoplanetas Confirmados: Factores Clave y Implicaciones para la Habitabilidad**

<font color = 'Black'>
<font size = 3>

**Autor**: [zicknezz13]  
**Última actualización**: [10/04/25]  

---

<font color = 'DodgerBlue'>
<font size = 6>
    
**Contexto Científico 🌌**

<font color = 'Black'>
<font size = 3>

- Los **exoplanetas** son planetas fuera de nuestro sistema solar. Desde 1992, se han descubierto miles, pero **solo el 30% están confirmados** (NASA Exoplanet Archive, 2023).

- **Problema**: Confirmar un exoplaneta requiere años de observación. ¿Podemos predecir qué candidatos tienen más probabilidades de ser confirmados usando datos de masa, órbita y estrellas anfitrionas?

---

<font color = 'DodgerBlue'>
<font size = 6>
    
**Objetivos**

<font color = 'Black'>
<font size = 3>

- Identificar **patrones** en **exoplanetas** **confirmados** vs. **candidatos** y predecir su **estado** (e.g., propiedades de la estrella anfitriona, etc.).
- Analizar indicadores de **habitabilidad planetaria** (e.g., temperatura, distancia orbital., etc.).
- Desarrollar un **modelo predictivo** para clasificar el estado de los planetas (**confirmado**/**candidato**).
- Crear un dashboard para visualizar la información.

---

<font color = 'DodgerBlue'>
<font size = 6>
    
**Preguntas clave**

<font color = 'Black'>
<font size = 3>

1. ¿Presentan los exoplanetas confirmados errores de medición de masa/radio significativamente menores?
2. ¿Se correlaciona la metalicidad de la estrella anfitriona (`star_metallicity`) con la probabilidad de confirmación de planetas?
3. ¿Qué métodos de detección (`detection_type`) son los más fiables para confirmar planetas?

---

<font color = 'DodgerBlue'>
<font size = 6>
    
**Data 📊**

<font color = 'Black'>
<font size = 3>

### Fuente
Los datos fueron obtenidos en el [Catálogo de Exoplanetas](https://exoplanet.eu/home/) en formato CSV. 
- Los datos se pueden descargar de manera gratuita. 
- Datos actualizados a la fecha de desacarga: [18/04/25].   

### Variables Relevantes
- **Planetas**: Masa (`mass(m_jup)`), radio (`radius(r_jup)`), periodo orbital, excentricidad.  
- **Estrellas**: Metalicidad (`star_metallicity`), temperatura efectiva (`star_teff`), distancia (`star_distance`).

> ⚠️ **Datos actualizados en el link del Catálogo de Exoplanetas.**

---

<font color = 'DodgerBlue'>
<font size = 6>
    
**Exploracion inicial del dataset con SQL**

<font color = 'DodgerBlue'>
<font size = 5>
    
- **Número de planetas en cada status y su porcentaje**

<font color = 'Black'>
<font size = 3>

Se muestra una tabla con el conteo de planetas por cada *planet_status* y su porcentaje.

| Planet_status | Count | Percentage |
|---------------|-------|------------|
| Unconfirmed   | 5     | 0.1        |
| Retracted     | 75    | 0.8        |
| Controversial | 70    | 0.7        |
| Candidate     | 2381  | 23.9       |
| Confirmed     | 7450  | 74.6       |

> ⚠️ **Los datos revelan que la mayoría de los planetas son 'confirmados' (74.6%), seguidos por los 'candidatos' (23.9%), con una proporción mínima en otras categorías. Estos dos *status* representan en 98.5% del total de exoplanetas descubiertos.**

La tabla anterior se obtuvo con el siguiente código de **PostgreSQL**:

```sql
-- Obtenemos el número de planetas que hay en cada status de los planetas y el porcentaje que representa.
SELECT planet_status, COUNT(*) AS conteo,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) AS percentage
FROM exoplanets_catalog
GROUP BY planet_status;
```

---

<font color = 'DodgerBlue'>
<font size = 5>
    
- **Porcentaje de valores nulos en algunas columnas del dataset**

<font color = 'Black'>
<font size = 3>

La siguiente tabla muestra el porcentaje de valores nulos que hay en cada una de las columnas seleccionadas.

| mass | radius | orbital_period | star_metallicity | temp_calculated | temp_measured | geometric_albedo | detection_type | molecules | star_distance | star_mass | star_sp_type | star_age | star_teff | star_detected_disc |
|------|--------|----------------|------------------|-----------------|---------------|------------------|----------------|-----------|---------------|-----------|--------------|----------|-----------|--------------------|
| 61.8 | 32.6   | 16.4           | 29.2             | 59.3            | 98.6          | 99.8             | 0.0            | 98.5      | 19.7          | 15.7      | 55.9         | 59.2     | 19.6      | 98.2               |

> ⚠️ **Se observa que la columna 'mass' presenta un 61.8% de valores nulos, lo que limita el conjunto de datos utilizable para análisis estadístico a solo un 38%.**

> ⚠️ **Las columnas 'temp_measured', 'geometric_albedo', 'star_detected_disc' y 'molecules' son las que contienen los mayores porcentajes de valores faltantes.**

> ⚠️ **Estos porcentajes proporcionan una visión general de la calidad del dataset y la cantidad de datos disponibles para realizar análisis y extraer conclusiones a través de la visualización.**

La tabla anterior se obtuvo con el siguiente código de **PostgreSQL**:

```sql
-- Obtenemos el porcentaje de valores nulos de algunas columnas:
SELECT
  ROUND((COUNT(*) - COUNT(mass)) * 100.0 / COUNT(*), 1) AS mass_null,
  ROUND((COUNT(*) - COUNT(radius)) * 100.0 / COUNT(*), 1) as radius_null,
  ROUND((COUNT(*) - COUNT(orbital_period)) * 100.0 / COUNT(*), 1) AS orbital_period_null,
  ROUND((COUNT(*) - COUNT(star_metallicity)) * 100.0 / COUNT(*), 1) AS metallicity_nulls,
  ROUND((COUNT(*) - COUNT(temp_calculated)) * 100.0 / COUNT(*), 1) as temp_calculated_null,
  ROUND((COUNT(*) - COUNT(temp_measured)) * 100.0 / COUNT(*), 1) as temp_measured_null,
  ROUND((COUNT(*) - COUNT(geometric_albedo)) * 100.0 / COUNT(*), 1) as geometric_albedo_null,
  ROUND((COUNT(*) - COUNT(detection_type)) * 100.0 / COUNT(*), 1) as detection_type_null,
  ROUND((COUNT(*) - COUNT(molecules)) * 100.0 / COUNT(*), 1) as molecules_null,
  ROUND((COUNT(*) - COUNT(star_distance)) * 100.0 / COUNT(*), 1) as star_distance_null,
  ROUND((COUNT(*) - COUNT(star_mass)) * 100.0 / COUNT(*), 1) as star_mass_null,
  ROUND((COUNT(*) - COUNT(star_sp_type)) * 100.0 / COUNT(*), 1) as star_sp_type_null,
  ROUND((COUNT(*) - COUNT(star_age)) * 100.0 / COUNT(*), 1) as star_age_null,
  ROUND((COUNT(*) - COUNT(star_teff)) * 100.0 / COUNT(*), 1) as teff_null,
  ROUND((COUNT(*) - COUNT(star_detected_disc)) * 100.0 / COUNT(*), 1) as star_detected_disc_null
FROM exoplanets_catalog;
```

---

<font color = 'DodgerBlue'>
<font size = 5>
    
- **Número de exoplanetas descubiertos por método, su porcentaje y masa promedio**

<font color = 'Black'>
<font size = 3>

La siguiente tabla muestra de manera resumida la cantidad de exoplanetas descubiertos por cada método, su porcentaje respecto al total de planetas descubiertos y la masa promedio de los exoplanetas. Algunos planetas se han descubierto mediante la combinacion de métodos.

| Detection_type              | Count | Percentege | Avg_Mass_Jup |
|-----------------------------|-------|------------|--------------|
| Primary Transit             | 6272  | 62.8       | 2.394        |
| Radial Velocity             | 1358  | 13.6       | 14.602       |
| Imaging                     | 1091  | 10.9       | 31.070       |
| Microlensing                | 326   | 3.3        | 7.034        |
| Timing                      | 207   | 2.1        | 24.384       |
| Other                       | 205   | 2.1        | 12.711       |
| Astrometry                  | 142   | 1.4        | 28.531       |
| Radial Velocity, Astrometry | 117   | 1.2        | 24.996       |
| Imaging, Astrometry         | 52    | 0.5        | 39.477       |
| Imaging, Other              | 48    | 0.5        | 27.376       |
| TTV                         | 43    | 0.4        | 2.572        |

> ⚠️ **El análisis de los métodos de detección revela que la técnica de 'tránsito primario' ha contribuido al 62% de los descubrimientos de exoplanetas, seguida por la 'velocidad radial' (13%) y el método de imagen con un (10%).**

> ⚠️ **Si bien el método de 'tránsito primario' es fundamental en la detección, los planetas así descubiertos tienden a ser considerablemente masivos, con un promedio de dos veces la masa de Júpiter.**

> ⚠️ **En contraste, los exoplanetas más masivos se han identificado mediante diversas técnicas, incluyendo 'imagen', 'timing', 'astrometría', 'velocidad radial y astrometría', e 'imagen y astrometría', según se indica en la tabla.**

La tabla anterior se obtuvo con el siguiente código de **PostgreSQL**:

```sql
-- Obtenemos una tabla con el número de planetas encontrados por detection_type, su porcentaje y masa promedio
SELECT detection_type, COUNT(*) AS conteo,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) AS percentage, AVG(mass) AS avg_mass_jup
FROM exoplanets_catalog
GROUP BY detection_type
ORDER BY COUNT(*) DESC;
```

---